# Solving a MQ problem with QAOA

Inspired by [1], we try solving the MQ problem with $5$ variables using the QAOAlgorithm.

The `article_lib.py` contains functions developed by Sergi Ramos-Calderer and Ruge Lin to implement different embeddings. The code is available online in the GitHub repository [qiboteam/mq-problem-quantum-annealing](https://github.com/qiboteam/mq-problem-quantum-annealing).

## Setup

In [139]:
## imports
import article_lib as article
import examples_nnf as nnf

import numpy as np
from scipy.sparse import dok_array 
from sympy import symbols

In [140]:
## qiskit's imports

from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler     
from qiskit_optimization import QuadraticProgram

## Building $\mathcal{H}$ as a `SparsePauliOp`

We start by building the Hamiltonian associated to our problem in a QUBO format using `article_lib.py`.

In [141]:
## building QUBO model
bits = 5

x5_sym = symbols(" ".join((f"x{i}" for i in range(1, bits+1))))
p5_sym, sol5 = nnf.example_5(*x5_sym)

direct = article.direct_embedding(bits, p5_sym, x5_sym)
H_sym = direct.create_hamiltonian()
sym = direct.get_symbols()

d_art = article.dwave_annealing(H_sym, bits, sym)
H_qubo, qubo_offset = d_art.symbolic_to_dwave(H_sym, d_art.get_symbol_num(sym))
H_qubo = dict(sorted(H_qubo.items()))

To define a cost function in Qiskit, we need the `linear` vector and the matrix $Q$ for the `quadratic` list of a list. 

In [142]:
## coefficients
linear = []
quadratic = dok_array((2**len(sym),2**len(sym))) 

count_linear = 0

for key in H_qubo.keys():

    if key[0] == key[1]:
        # linear
        while len(linear) != key[0]:
            linear.append(0)
        linear.append(H_qubo[key])
    else:
        # quadratic
        quadratic[key[0],key[1]] = H_qubo[key]

We first create the associated QUBO as a `QuadraticProgram` Qiskit class

In [143]:
qiskit_qubo = QuadraticProgram('QUBO model from Article')

qubo_x = [f'x{i}' for i in range(bits)]
qubo_anc = [f'a{i}' for i in range(5)]
qubo_var = qubo_x + qubo_anc

## variables
for var in qubo_var:
    qiskit_qubo.binary_var(var)

qiskit_qubo.variables

[<Variable: x0 (binary)>,
 <Variable: x1 (binary)>,
 <Variable: x2 (binary)>,
 <Variable: x3 (binary)>,
 <Variable: x4 (binary)>,
 <Variable: a0 (binary)>,
 <Variable: a1 (binary)>,
 <Variable: a2 (binary)>,
 <Variable: a3 (binary)>,
 <Variable: a4 (binary)>]

In [144]:
## cost function
qiskit_qubo.minimize(constant=qubo_offset, linear=linear, quadratic=quadratic)
print(qiskit_qubo.prettyprint())

Problem name: QUBO model from Article

Minimize
  8*a0*a2 - 6*a0*a3 - 6*a0*a4 - 18*a1*a2 - 30*x0*a0 + 2*x0*a1 - 2*x0*a2
  + 4*x0*a3 + 2*x0*a4 + 15*x0*x1 - 2*x0*x2 - 2*x0*x3 - x0*x4 - 30*x1*a0
  + 2*x1*a1 - 6*x1*a2 + 2*x1*a3 + 2*x1*a4 - x1*x4 + 4*x2*a0 + 9*x2*a1 - 18*x2*a2
  - 16*x2*a3 - 14*x2*a4 + 8*x2*x3 + 7*x2*x4 + 2*x3*a0 - 26*x3*a1 - 16*x3*a3
  + 13*x3*x4 - 26*x4*a1 - 14*x4*a4 + 43*a0 + 39*a1 + 27*a2 + 23*a3 + 22*a4
  + 2*x0 + 2

Subject to
  No constraints

  Binary variables (10)
    x0 x1 x2 x3 x4 a0 a1 a2 a3 a4



and then we turn it into an Ising model, which is the correct object type the minimizer takes as input.

In [145]:
## to Ising
ising_model, ising_offset = qiskit_qubo.to_ising()
print(f"offset: {ising_offset}")
print("operator:")
print(ising_model)

offset: 41.5
operator:
SparsePauliOp(['IIIIIIIIIZ', 'IIIIZIIIII', 'IIIZIIIIII', 'IIZIIIIIII', 'IZIIIIIIII', 'ZIIIIIIIII', 'IIIIIIIIZZ', 'IIIIIIIIZI', 'IIIIIIIZIZ', 'IIIIIIIZII', 'IIIIIIZIIZ', 'IIIIIIZIII', 'IIIIIIZZII', 'IIIIIZIIIZ', 'IIIIIZIIII', 'IIIIIZIIZI', 'IIIIIZIZII', 'IIIIIZZIII', 'IIIIZIIIIZ', 'IIIIZIIIZI', 'IIIIZIIZII', 'IIIIZIZIII', 'IIIZIIIIIZ', 'IIIZIIIIZI', 'IIIZIIIZII', 'IIIZIIZIII', 'IIIZIZIIII', 'IIZIIIIIIZ', 'IIZIIIIIZI', 'IIZIIIIZII', 'IIZIZIIIII', 'IIZZIIIIII', 'IZIIIIIIIZ', 'IZIIIIIIZI', 'IZIIIIIZII', 'IZIIIIZIII', 'IZIIZIIIII', 'ZIIIIIIIIZ', 'ZIIIIIIIZI', 'ZIIIIIIZII', 'ZIIIIZIIII', 'ZIIIZIIIII'],
              coeffs=[ 2.5 +0.j, -7.  +0.j, -5.25+0.j, -4.5 +0.j, -3.5 +0.j, -3.5 +0.j,
  3.75+0.j,  4.  +0.j, -0.5 +0.j,  5.5 +0.j, -0.5 +0.j,  5.25+0.j,
  2.  +0.j, -0.25+0.j,  5.5 +0.j, -0.25+0.j,  1.75+0.j,  3.25+0.j,
 -7.5 +0.j, -7.5 +0.j,  1.  +0.j,  0.5 +0.j,  0.5 +0.j,  0.5 +0.j,
  2.25+0.j, -6.5 +0.j, -6.5 +0.j, -0.5 +0.j, -1.5 +0.j, -4.5 +0.j,
  2.  +0.j, -4.5 

## QAOA

To implement the QAOAlgorithm we rely on Qiskit homonimous class [2], namely:

<center>

`qiskit.algorithms.minimum_eigensolvers.QAOA(sampler, optimizer, *, reps=1, initial_state=None, mixer=None, initial_point=None, aggregation=None, callback=None)`

</center>

We also define a custom `callback` function to track the evolution of the ansatz parameters.

In [146]:
def my_callback(evaluation_count, optimizer_parameters, estimated_value, metadata):

    global callback_data, cost

    callback_data.append(optimizer_parameters)
    cost.append(estimated_value)
    
    #print(optimizer_parameters)

In [147]:
## hyperparameters
p = 1
initial_point = [np.random.rand() for _ in range(2*p)]
callback_data = []
cost = []

## QAOA class
qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(), reps=p, initial_point=initial_point, callback=my_callback)

To run our optimization routine we call the `compute_minimum_eigenvalue()` method of the QAOA class:

In [148]:
## minimizing
qaoa_result = qaoa_mes.compute_minimum_eigenvalue(ising_model)

As for the final results, we are interested in the best measurement and the number of function evaluations:

In [149]:
print(f'Best measurement:{qaoa_result.best_measurement}')
print(f'Number of function evaluations: {qaoa_result.cost_function_evals}')

Best measurement:{'state': 51, 'bitstring': '0000110011', 'value': (-41.5+0j), 'probability': 0.0042611152743709}
Number of function evaluations: 228


The best measurement is a dictionary with a few informations, but we only need the `bitstring` one to retrieve the solution to our MQ problem. Specifically, we need only the measured bit associated to the first $5$ qubits.

Please notice that `bitstring` is reversed since Qiskit uses the little-endian convention.

In [150]:
ph_string = qaoa_result.best_measurement['bitstring'][::-1]
ph_prob = qaoa_result.best_measurement['probability']
print(f'Best measurement: {ph_string}')
print(f'obtained with probability {ph_prob}')

print('\n------------------------------------------------------------------------------\n')

knwon_sol = ''.join(str(i) for i in sol5[0])
print(f'Solution found: {ph_string[:bits]}')
print(f'Known solution(s): {knwon_sol}')

Best measurement: 1100110000
obtained with probability 0.0042611152743709

------------------------------------------------------------------------------

Solution found: 11001
Known solution(s): 11001


The QAOA found the correct solution!

## Simulating a noisy run

Since running on a real QPU would exceed the available monthly usage, we fall back to simulating noise on our computation. Hence, we start by setting up a noise model on the fake backend `FakeWashingtonV2`.

In [151]:
from qiskit_ibm_runtime import QiskitRuntimeService, Options
from qiskit.providers.fake_provider import FakeWashingtonV2
from qiskit.primitives import BackendSampler
from qiskit_aer.noise import NoiseModel
 
service = QiskitRuntimeService()

backend = FakeWashingtonV2()                    ## fake backend
noise_model = NoiseModel.from_backend(backend)  ## + noise

options = Options()
options.resilience_level = 2
options.optimization_level = 3
options.simulator.set_backend(backend)
options_dict = options.__dict__
 
sampler = BackendSampler(backend, options_dict) ## fake Sampler            

Since the sampler is not the Qiskit primitive but a fake one tailored to our fake backend, the callback is not an available option and it will not be implemented.

In [152]:
## hyperparameters
p = 1
initial_point = [np.random.rand() for _ in range(2*p)]

## QAOA class
qaoa_mes_noise = QAOA(sampler=sampler, optimizer=COBYLA(), reps=p, initial_point=initial_point)

## minimizing
qaoa_noise_result = qaoa_mes_noise.compute_minimum_eigenvalue(ising_model)

In [153]:
print(f'Best measurement:{qaoa_noise_result.best_measurement}')
print(f'Number of function evaluations: {qaoa_noise_result.cost_function_evals}')

Best measurement:{'state': 51, 'bitstring': '0000110011', 'value': (-41.5+0j), 'probability': 0.00390625}
Number of function evaluations: 30


Once again, we focus on the `bitstring` variable to see if noise hindered our algorithm.

In [154]:
ph_string = qaoa_noise_result.best_measurement['bitstring'][::-1]
ph_prob = qaoa_noise_result.best_measurement['probability']
print(f'Best measurement: {ph_string}')
print(f'obtained with probability {ph_prob}')

print('\n------------------------------------------------------------------------------\n')

knwon_sol = ''.join(str(i) for i in sol5[0])
print(f'Solution found: {ph_string[:bits]}')
print(f'Known solution(s): {knwon_sol}')

Best measurement: 1100110000
obtained with probability 0.00390625

------------------------------------------------------------------------------

Solution found: 11001
Known solution(s): 11001


It didn't! The noise mitigation worked perfectly and we got the correct answer in this case too.

# References
[1] ["Solving systems of Boolean multivariate equations with quantum annealing"](https://arxiv.org/abs/2111.13224) by Sergi Ramos-Calderer, Carlos Bravo-Prieto, Ruge Lin, Emanuele Bellini, Marc Manzano, Najwa Aaraj, and José I. Latorre <br>
[2] [QAOA Qiskit](https://qiskit-community.github.io/qiskit-algorithms/stubs/qiskit_algorithms.QAOA.html)

In [169]:
import qiskit.tools.jupyter
%qiskit_version_table